# Librerías

In [22]:
# import libraries
import pandas as pd
import os
from io import StringIO

# Cargar y unir datasets

## Cargar todos los csv


In [23]:
DATA_DIR = "Datasets"

FILES = {
    "municipios_base": "municipios_madrid_menores_50000.csv",
    "centros_salud": "recursos-y-asistencia-sanitaria.-centros-por-tipo-de-centro.-municipios.csv",
    "farmacias": "recursos-sanitarios.-farmacias.csv",
    "google_places": "google_places_municipios.csv",
    "paro_total": "total-paro-registrado.csv",
    "paro_por100": "paro-registrado-por-100-habitantes.csv",
    "cultura": "numero de intereses culturales.csv",
    "distancia": "distancia-a-la-capital_2016.csv",
    "vivienda": "IPVA 2023 VIVIENDA.csv",
    "afiliados": "afiliados-por-lugar-de-residencia-y-actividad.-municipios.csv",
    "alumnos": "alumnos-no-universitarios-del-regimen-general-matriculados-en-centros-escolares-por-tipo-de-cent.csv"
}

# === FUNCIONES ===
def cargar_csv_detectando_sep(path):
    """Carga CSV detectando separador y codificación automáticamente."""
    for sep in [",", ";"]:
        try:
            df_test = pd.read_csv(path, sep=sep, nrows=5)
            if len(df_test.columns) > 1:
                full_df = pd.read_csv(path, sep=sep)
                return full_df, sep, None
        except Exception:
            continue
    for encoding in ["utf-8", "ISO-8859-1"]:
        for sep in [",", ";"]:
            try:
                full_df = pd.read_csv(path, sep=sep, encoding=encoding)
                if len(full_df.columns) > 1:
                    return full_df, sep, encoding
            except Exception:
                continue
    return None, None, None

def explorar_dataset(nombre, path):
    """Carga y muestra información básica de un dataset."""
    print(f"\n📂 {nombre} — {path}")
    df, sep, encoding = cargar_csv_detectando_sep(path)
    
    if df is None:
        print("❌ No se pudo cargar el archivo correctamente.")
        return None
    
    print(f"✔️ Separador: '{sep}' | Codificación: '{encoding}' | Filas: {len(df)} | Columnas: {len(df.columns)}")
    print("🧾 Columnas:", list(df.columns)[:8], "..." if len(df.columns) > 8 else "")
    print(df.head(3))
    return df

def inspeccionar_texto(path, n=8):
    """Muestra algunas líneas del archivo en bruto para inspección manual."""
    print(f"\n🔎 Inspeccionando texto bruto de {path}:")
    try:
        with open(path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                print(line.strip())
                if i >= n:
                    break
    except UnicodeDecodeError:
        print("⚠️ Error de codificación UTF-8, probando ISO-8859-1...")
        with open(path, "r", encoding="ISO-8859-1") as f:
            for i, line in enumerate(f):
                print(line.strip())
                if i >= n:
                    break

# --- CARGA PRINCIPAL ---
def main():
    dfs = {}
    for name, filename in FILES.items():
        path = os.path.join(DATA_DIR, filename)
        if not os.path.exists(path):
            print(f"⚠️ Archivo no encontrado: {path}")
            continue
        df = explorar_dataset(name, path)
        if df is not None:
            dfs[name] = df
        else:
            inspeccionar_texto(path)
    print("\n✅ Exploración completa. DataFrames cargados:", list(dfs.keys()))
    return dfs

if __name__ == "__main__":
    dfs = main()



📂 municipios_base — Datasets\municipios_madrid_menores_50000.csv
✔️ Separador: ',' | Codificación: 'None' | Filas: 155 | Columnas: 6
🧾 Columnas: ['cod_municipio', 'municipio', 'latitud', 'longitud', 'altitud', 'poblacion'] 
   cod_municipio          municipio   latitud  longitud    altitud  poblacion
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0

📂 centros_salud — Datasets\recursos-y-asistencia-sanitaria.-centros-por-tipo-de-centro.-municipios.csv
✔️ Separador: ',' | Codificación: 'None' | Filas: 573 | Columnas: 7
🧾 Columnas: ['Año', 'Tipo territorio', 'Código territorio', 'Territorio', 'Tipo', 'Medida', 'Valor'] 
    Año      Tipo territorio  Código territorio           Territorio  \
0  2024  Comunidad de Madrid                NaN  Comunidad de Madrid   
1  2024           Municipios               14

El archivo "numero de intereses culturales.csv" no sigue el formato de un csv regular, presenta 22 columnas pero hay un punto y coma extra en el final de cada fila. Adem'as, las conas son para numeros decimales y la primera y segunda fila son descriptivas. Por lo que se eliminaría las primeras filas, limpiar texto quitando el ; final y luego las cuatro primeras columnas los nombraríamos: "Tipo territorio", "Código territorio", "Territorio". A esto le añadimos las columnas años que van desde 2006 hasta el 2024. 

In [24]:
def cargar_cultura_limpia(path):
    """Carga y limpia el dataset de cultura (corrige ';' extra y pasa de ancho a largo)."""
    with open(path, "rb") as f:
        raw = f.read()

    # eliminar BOM si existe
    if raw.startswith(b'\xef\xbb\xbf'):
        raw = raw[len(b'\xef\xbb\xbf'):]
    text = raw.decode("ISO-8859-1", errors="replace")

    # limpiar saltos de línea
    lines = [ln.rstrip("\r\n") for ln in text.splitlines()]
    # saltar dos primeras líneas descriptivas
    data_lines = lines[2:]

    cleaned = []
    for ln in data_lines:
        ln = ln.replace('\ufeff', '').replace('\u200b', '').strip()
        # eliminar cualquier ';' extra al final
        while ln.endswith(';'):
            ln = ln[:-1]
        cleaned.append(ln)

    # detectar columnas esperadas (primer fila válida)
    expected_cols = None
    for ln in cleaned:
        parts = ln.split(';')
        if len(parts) >= 3:
            expected_cols = len(parts)
            break

    if expected_cols is None:
        raise ValueError("No se pudo inferir el número de columnas del fichero de cultura.")

    normalized = []
    for ln in cleaned:
        parts = ln.split(';')
        # eliminar columnas vacías extra al final
        while len(parts) > expected_cols and parts[-1] == '':
            parts = parts[:-1]
        # recortar o rellenar según corresponda
        if len(parts) > expected_cols:
            parts = parts[:expected_cols-1] + [';'.join(parts[expected_cols-1:])]
        if len(parts) < expected_cols:
            parts += [''] * (expected_cols - len(parts))
        normalized.append(';'.join(parts))

    # encabezado correcto
    years = [str(y) for y in range(2006, 2025)]
    header = ["Tipo territorio", "Código territorio", "Territorio"] + years
    csv_text = ";".join(header) + "\n" + "\n".join(normalized)

    # leer con pandas
    df = pd.read_csv(StringIO(csv_text), sep=';', decimal=',', dtype=str, engine='python')

    # quedarnos con los municipios
    df = df[df["Tipo territorio"] == "Municipios"]

    # pasar a formato largo
    df_long = df.melt(
        id_vars=["Código territorio", "Territorio"],
        var_name="Año",
        value_name="Valor"
    )

    # limpiar tipos
    df_long["Año"] = pd.to_numeric(df_long["Año"], errors="coerce").astype("Int64")
    df_long["Valor"] = (
        df_long["Valor"]
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df_long["Valor"] = pd.to_numeric(df_long["Valor"], errors="coerce")

    print(f"✅ Dataset 'cultura' cargado correctamente: {df_long.shape}")
    print(df_long.head(200))
    return df_long


# Ejemplo de uso
if __name__ == "__main__":
    df_cultura = cargar_cultura_limpia("Datasets/numero de intereses culturales.csv")

dfs["cultura"] = df_cultura

✅ Dataset 'cultura' cargado correctamente: (3580, 4)
    Código territorio             Territorio   Año  Valor
0                0014           Acebeda (La)  <NA>    NaN
1                0029                Ajalvir  <NA>    NaN
2                0035      Alameda del Valle  <NA>    NaN
3                0040            Ãlamo (El)  <NA>    NaN
4                0053     AlcalÃ¡ de Henares  <NA>    NaN
..                ...                    ...   ...    ...
195              0170                 Batres  2006    2.0
196              0186  Becerril de la Sierra  2006    0.0
197              0199       Belmonte de Tajo  2006    0.0
198              0210          Berrueco (El)  2006    1.0
199              0203     Berzosa del Lozoya  2006    0.0

[200 rows x 4 columns]


## Limpieza y tranformación antes de unirse

In [25]:
def pipeline_general(df, year_col="Año", code_col="Código territorio", value_col="Valor", last_year=True, to_int=True, rename_value=None):
    """Función general para transformar datasets con estructura común"""
    df = df.copy()
    
    # Filtrar por último año
    if last_year and year_col in df.columns:
        # garantizar que el año se compare como numérico (puede haber NaNs)
        df_years = pd.to_numeric(df[year_col], errors="coerce")
        max_year = df_years.max()
        df = df[df_years == max_year]
    
    # Convertir código de municipio a entero nullable (soporta NA)
    if code_col in df.columns:
        df["cod_municipio"] = pd.to_numeric(df[code_col], errors="coerce").astype("Int64")
    
    # Renombrar columna de valor (si se pide)
    if rename_value and value_col in df.columns:
        df = df.rename(columns={value_col: rename_value})
    
    # determinar nombre final de la columna de valor
    target_value_col = rename_value if rename_value else value_col
    
    # Convertir columna valor a numérica (soporta coma como decimal si ya limpia; en caso contrario intenta conversión directa)
    if target_value_col in df.columns:
        # si es object, intentar limpiar separadores comunes (puntos miles, comas decimales)
        if df[target_value_col].dtype == object:
            # Reemplazos no destructivos: quitar espacios y convertir comas a puntos
            df[target_value_col] = (
                df[target_value_col]
                .astype(str)
                .str.replace(r"\s+", "", regex=True)
                .str.replace(".", "", regex=False)  # eliminar separadores de miles
                .str.replace(",", ".", regex=False)  # coma decimal -> punto
            )
        df[target_value_col] = pd.to_numeric(df[target_value_col], errors="coerce")
    
    return df

# --- Función específica para vivienda (más robusta) ---
def pipeline_vivienda(df):
    df = df.copy()

    # Si ya está procesado (columnas esperadas), devolver versión normalizada
    if {"cod_postal", "municipio", "IPVA"}.issubset(df.columns):
        df["cod_postal"] = pd.to_numeric(df["cod_postal"], errors="coerce").astype("Int64")
        df["IPVA"] = pd.to_numeric(df["IPVA"], errors="coerce")
        df["municipio"] = df["municipio"].astype(str).str.strip()
        return df[["cod_postal", "municipio", "IPVA"]].reset_index(drop=True)

    # Localizar columna que contiene municipio (acepta varias variantes)
    muni_col = None
    for c in ["Municipio", "Territorio", "municipio", "territorio"]:
        if c in df.columns:
            muni_col = c
            break
    if muni_col is None:
        raise KeyError("No se encontró columna de municipio. Buscando 'Municipio' o 'Territorio'.")

    # Localizar columna de periodo/año si existe
    period_col = None
    for c in ["Periodo", "Año", "Year"]:
        if c in df.columns:
            period_col = c
            break
    # Filtrar por último año ignorando valores nulos en periodo
    if period_col is not None:
        periodo_num = pd.to_numeric(df[period_col], errors="coerce")
        valid_periods = periodo_num.dropna()
        if not valid_periods.empty:
            last_year = valid_periods.max()
            df = df[periodo_num == last_year]

    # Normalizar texto de la columna municipio y separar código y nombre
    s = df[muni_col].astype(str).fillna("").str.strip()
    # reemplazar guiones largos por espacio y limpiar espacios extra
    s_clean = s.str.replace(r"[-–—]+", " ", regex=True).str.strip()
    parts = s_clean.str.split(r"\s+", n=1, expand=True)

    # Extraer dígitos iniciales como código postal/código municipio cuando existan
    first_part_digits = parts[0].str.extract(r"(\d+)", expand=False)
    df["cod_postal"] = pd.to_numeric(first_part_digits, errors="coerce").astype("Int64")

    # Si no hay segunda parte, usar la primera (útil si ya solo contiene nombre)
    df["municipio"] = parts[1].where(parts[1].notna(), parts[0]).astype(str).str.strip()

    # Localizar columna de valor (Total/IPVA u otras variantes)
    value_col = None
    for c in ["Total", "IPVA", "Valor", "Importe"]:
        if c in df.columns:
            value_col = c
            break
    if value_col is None:
        # intentar usar la primera columna numérica razonable
        numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
        # excluir columnas que claramente no son el valor buscado
        for excl in ["cod_postal"]:
            if excl in numeric_cols:
                numeric_cols.remove(excl)
        if numeric_cols:
            value_col = numeric_cols[0]

    if value_col is None:
        raise KeyError("No se encontró columna de valor (Total/IPVA/Valor) en el dataset de vivienda.")

    # Limpiar formato del valor y convertir a numérico
    df["IPVA"] = (
        df[value_col]
        .astype(str)
        .str.replace(r"\s+", "", regex=True)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df["IPVA"] = pd.to_numeric(df["IPVA"], errors="coerce")

    return df[["cod_postal", "municipio", "IPVA"]].reset_index(drop=True)

# --- Función específica para cultura (maneja años nulos) ---
def pipeline_cultura(df):
    df = df.copy()
    # Filtrar por último año ignorando valores nulos en 'Año'
    if "Año" in df.columns:
        year_num = pd.to_numeric(df["Año"], errors="coerce")
        valid_years = year_num.dropna()
        if not valid_years.empty:
            last_year = valid_years.max()
            df = df[year_num == last_year]
        # si no hay años válidos, no filtramos (mantenemos todo)
    # convertir código a entero nullable (si existe)
    if "Código territorio" in df.columns:
        df["cod_municipio"] = pd.to_numeric(df["Código territorio"], errors="coerce").astype("Int64")
    else:
        # intentar encontrar columna parecida si cambia el nombre
        for c in df.columns:
            if "codigo" in c.lower() or "territorio" in c.lower():
                df["cod_municipio"] = pd.to_numeric(df[c], errors="coerce").astype("Int64")
                break
    # asegurar Valor numérico si existe
    if "Valor" in df.columns:
        df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")
    return df[["cod_municipio", "Valor"]]

# Aplicar pipelines solo si existen las claves en dfs (evita KeyError)
if "centros_salud" in dfs:
    dfs["centros_salud"] = pipeline_general(dfs["centros_salud"], year_col="Año", code_col="Código territorio", value_col="Valor", rename_value="centros_salud")
if "farmacias" in dfs:
    dfs["farmacias"] = pipeline_general(dfs["farmacias"], rename_value="farmacias")
if "paro_total" in dfs:
    dfs["paro_total"] = pipeline_general(dfs["paro_total"], rename_value="paro_total")
if "paro_por100" in dfs:
    dfs["paro_por100"] = pipeline_general(dfs["paro_por100"], rename_value="paro_100")
if "distancia" in dfs:
    dfs["distancia"] = pipeline_general(dfs["distancia"], rename_value="distancia_capital")
if "vivienda" in dfs:
    dfs["vivienda"] = pipeline_vivienda(dfs["vivienda"])
if "afiliados" in dfs:
    dfs["afiliados"] = pipeline_general(dfs["afiliados"], rename_value="afiliados")
if "alumnos" in dfs:
    dfs["alumnos"] = pipeline_general(dfs["alumnos"], rename_value="alumnos")
if "cultura" in dfs:
    dfs["cultura"] = pipeline_cultura(dfs["cultura"])

In [26]:
# ✅ Revisar que todo se haya transformado correctamente
for name, df in dfs.items():
    print(f"\n{name.upper()} — Filas: {len(df)} | Columnas: {df.shape[1]}")
    print(df.head(5))


MUNICIPIOS_BASE — Filas: 155 | Columnas: 6
   cod_municipio          municipio   latitud  longitud    altitud  poblacion
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0
3             40         Álamo (El)  40.22972 -3.992688   606.2238    10413.0
4             88   Aldea del Fresno  40.32399 -4.202217   476.7994     3422.0

CENTROS_SALUD — Filas: 573 | Columnas: 8
    Año      Tipo territorio  Código territorio           Territorio  \
0  2024  Comunidad de Madrid                NaN  Comunidad de Madrid   
1  2024           Municipios               14.0         Acebeda (La)   
2  2024           Municipios               29.0              Ajalvir   
3  2024           Municipios               35.0    Alameda del Valle   
4  2024           Municipios               40.0           Álamo (El)   

             

## Unión de datasets: DATASET COMPLETO

In [27]:
# === FUNCIONES AUXILIARES ===

def preparar_afiliados(df):
    """Convierte afiliados (one-to-many) a formato ancho por rama de actividad.
    Normaliza la columna 'Rama de actividad' (minúsculas, sin espacios ni caracteres inválidos)
    antes del pivot.
    """
    df = df.copy()
    # Normalizar texto
    df["Rama de actividad"] = df["Rama de actividad"].astype(str).str.strip().str.lower()
    # Reemplazar espacios por guiones bajos y eliminar caracteres no alfanuméricos/guión bajo
    df["rama_norm"] = (
        df["Rama de actividad"]
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w]", "", regex=True)
    )
    # Asegurar que 'afiliados' es numérico
    df["afiliados"] = pd.to_numeric(df.get("afiliados"), errors="coerce").fillna(0)
    # Pivotear
    df_pivot = df.pivot_table(
        index="cod_municipio",
        columns="rama_norm",
        values="afiliados",
        aggfunc="sum",
        fill_value=0,
    ).reset_index()
    # Renombrar columnas de salida
    df_pivot.columns = ["cod_municipio"] + [f"afiliados_{c}" for c in df_pivot.columns[1:]]
    return df_pivot

def preparar_alumnos(df):
    """Agrupa por municipio para obtener el total de alumnos no universitarios."""
    df = df.copy()
    df["alumnos"] = pd.to_numeric(df["alumnos"], errors="coerce")
    df_agg = df.groupby("cod_municipio", as_index=False)["alumnos"].sum()
    df_agg = df_agg.rename(columns={"alumnos": "alumnos_total"})
    return df_agg

def preparar_cultura(df):
    df = df.copy()
    df = df.rename(columns={"Valor": "cultura"})
    return df[["cod_municipio", "cultura"]]

def preparar_centros_salud(df):
    df = df.copy()
    # Filtrar solo las filas donde la columna 'Tipo' indique 'Centros de salud'
    if "Tipo" in df.columns:
        df = df[df["Tipo"].astype(str).str.strip().str.lower() == "centros de salud"].copy()
    df = df.rename(columns={"centros_salud": "centros_salud_10mil"})
    return df[["cod_municipio", "centros_salud_10mil"]]

def preparar_farmacias(df):
    df = df.copy()
    df = df.rename(columns={"farmacias": "farmacias_10mil"})
    return df[["cod_municipio", "farmacias_10mil"]]

def preparar_vivienda(df):
    df = df.copy()
    df = df.rename(columns={"municipio": "municipio", "IPVA": "IPVA"})
    df["municipio"] = df["municipio"].str.strip().str.lower()
    return df[["municipio", "IPVA"]]

def preparar_municipios_base(df):
    df = df.copy()
    return df[["cod_municipio", "municipio", "latitud", "longitud", "altitud", "poblacion"]]

# === PIPELINE DE UNIÓN ===
def unir_todos(dfs, output_path="df_final.csv"):
    # Base: municipios
    df_final = preparar_municipios_base(dfs["municipios_base"])

    def merge(df_dest, name, how="left", on="cod_municipio"):
        if name not in dfs:
            print(f"⚠️ {name} no encontrado, se omite.")
            return df_dest

        print(f"🧩 Uniendo {name}...")
        df_src = dfs[name]
        result = df_dest.merge(df_src, on=on, how=how, suffixes=("", f"_{name}"))

        # Si se crean columnas municipio_x / municipio_y, mantener solo la de municipios_base
        for col in ["municipio_y", f"municipio_{name}"]:
            if col in result.columns:
                result.drop(columns=[col], inplace=True)
        if "municipio_x" in result.columns:
            result.rename(columns={"municipio_x": "municipio"}, inplace=True)

        return result

    # Preparar datasets específicos
    if "centros_salud" in dfs:
        dfs["centros_salud"] = preparar_centros_salud(dfs["centros_salud"])
    if "farmacias" in dfs:
        dfs["farmacias"] = preparar_farmacias(dfs["farmacias"])
    if "cultura" in dfs:
        dfs["cultura"] = preparar_cultura(dfs["cultura"])
    if "afiliados" in dfs:
        dfs["afiliados"] = preparar_afiliados(dfs["afiliados"])
    if "alumnos" in dfs:
        dfs["alumnos"] = preparar_alumnos(dfs["alumnos"])
    if "vivienda" in dfs:
        dfs["vivienda"] = preparar_vivienda(dfs["vivienda"])

    # Merge progresivo por cod_municipio
    for name in [
        "centros_salud",
        "farmacias",
        "google_places",
        "paro_total",
        "paro_por100",
        "distancia",
        "afiliados",
        "alumnos",
        "cultura",
    ]:
        df_final = merge(df_final, name)

    # Unir vivienda por municipio (texto)
    if "vivienda" in dfs:
        df_viv = dfs["vivienda"]
        df_final["municipio_norm"] = df_final["municipio"].str.strip().str.lower()
        df_viv["municipio"] = df_viv["municipio"].astype(str).str.strip().str.lower()

        df_final = df_final.merge(
            df_viv,
            left_on="municipio_norm",
            right_on="municipio",
            how="left"
        )
        df_final.drop(columns=["municipio_norm", "municipio_y"], inplace=True, errors="ignore")
        df_final.rename(columns={"municipio_x": "municipio"}, inplace=True)

    # Guardar resultado
    df_final.to_csv(output_path, index=False)
    print(f"\n✅ DataFrame final guardado en: {output_path}")
    print(f"Filas: {len(df_final)} | Columnas: {df_final.shape[1]}")
    return df_final

# Unir y ver resultado final
df_final = unir_todos(dfs)
# Definir columnas finales a conservar
final_columns = [
    "cod_municipio", "municipio", "latitud", "longitud", "altitud", "poblacion",
    "centros_salud_10mil", "farmacias_10mil",
    "n_gym", "gym_total_reviews", "gym_weighted_avg_rating", "gym_rating_min", "gym_rating_max",
    "n_school", "school_total_reviews", "school_weighted_avg_rating", "school_rating_min", "school_rating_max",
    "n_transport", "transport_total_reviews", "transport_weighted_avg_rating", "transport_rating_min", "transport_rating_max",
    "n_restaurant", "restaurant_total_reviews", "restaurant_weighted_avg_rating", "restaurant_rating_min", "restaurant_rating_max",
    "n_pharmacy", "pharmacy_total_reviews", "pharmacy_weighted_avg_rating", "pharmacy_rating_min", "pharmacy_rating_max",
    "paro_total", "paro_100", "distancia_capital",
    "alumnos_total", "cultura", "IPVA"
]

# Filtrar solo las columnas que queremos conservar
cols_to_keep = [c for c in df_final.columns if c in final_columns or c.startswith("afiliados_")]
df_final = df_final[cols_to_keep]
print(f"\n=== DATAFRAME FINAL UNIDO ===\nFilas: {len(df_final)} | Columnas: {df_final.shape[1]}")
print(df_final.head(10))
print(df_final.columns.tolist())


🧩 Uniendo centros_salud...
🧩 Uniendo farmacias...
🧩 Uniendo google_places...
🧩 Uniendo paro_total...
🧩 Uniendo paro_por100...
🧩 Uniendo distancia...
🧩 Uniendo afiliados...
🧩 Uniendo alumnos...
🧩 Uniendo cultura...

✅ DataFrame final guardado en: df_final.csv
Filas: 155 | Columnas: 64

=== DATAFRAME FINAL UNIDO ===
Filas: 155 | Columnas: 46
   cod_municipio          municipio   latitud  longitud    altitud  poblacion  \
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0   
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0   
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0   
3             40         Álamo (El)  40.22972 -3.992688   606.2238    10413.0   
4             88   Aldea del Fresno  40.32399 -4.202217   476.7994     3422.0   
5             91             Algete  40.59642 -3.497902   712.8972    21134.0   
6            105          Alpedrete  40.65875 -4.023713   916.7029    15655.0   
7         

In [28]:
# Guardar CSV
# output_path = "Datasets/df_final.csv"
# df_final.to_csv(output_path, index=False, encoding="utf-8-sig")

# EDA (df_final)